In [4]:
%%bash
pip install sklearn
pip install tqdm

You are using pip version 8.1.1, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 8.1.1, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
import os, sys
sys.path.append('utils/')
import matplotlib.pyplot as plt
import numpy as np
import cPickle as pkl
from sklearn.decomposition import PCA
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import KFold
import pandas as pd

from keras.layers import Input, Dense, Dropout, Flatten, Lambda
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.models import Model
# from keras.utils.visualize_util import plot
from keras.utils import np_utils
import keras.backend as K

from sklearn.manifold import TSNE

from keras.engine.training import make_batches

from flip_gradient_keras import GradientReversalLayer
from utils import *
from evaluation import compute_ks, roc_auc_truncated
from tqdm import tqdm, trange

SEED = 5

pd.set_option('display.max_columns', 100)

%matplotlib inline

# Preparing data

In [7]:
label_prediction = pd.read_csv('../datasets/training.csv.zip').drop('id', 1)
label_prediction = pd.concat([label_prediction, label_prediction]).sample(frac=1., random_state=SEED)
features = label_prediction.drop('signal', 1).columns
print label_prediction.shape
label_prediction.head()

(135106, 50)


,LifeTime,dira,FlightDistance,FlightDistanceError,IP,IPSig,VertexChi2,pt,DOCAone,DOCAtwo,DOCAthree,IP_p0p2,IP_p1p2,isolationa,isolationb,isolationc,isolationd,isolatione,isolationf,iso,CDF1,CDF2,CDF3,ISO_SumBDT,p0_IsoBDT,p1_IsoBDT,p2_IsoBDT,p0_track_Chi2Dof,p1_track_Chi2Dof,p2_track_Chi2Dof,p0_IP,p1_IP,p2_IP,p0_IPSig,p1_IPSig,p2_IPSig,p0_pt,p1_pt,p2_pt,p0_p,p1_p,p2_p,p0_eta,p1_eta,p2_eta,SPDhits,production,signal,mass,min_ANNmuon
21840,0.001173,0.999793,7.309732,0.225475,0.144760,8.930744,4.965856,4945.793457,0.019389,0.015251,0.037411,0.204333,0.249398,1,5,3,1.0,2.0,2.0,5.0,1.000000,0.643805,0.270755,-0.423511,-0.131208,-0.115879,-0.176424,1.225634,1.368773,1.647525,0.339441,0.228714,0.454121,10.899674,11.228765,24.274082,885.767273,1630.131714,2696.752686,8884.096680,12979.206055,17545.503906,2.996217,2.763868,2.559938,572,-99,0,1885.391968,0.464575
56053,0.001524,0.999998,12.253985,0.171148,0.026300,2.110847,7.534899,6609.697754,0.041670,0.008835,0.026718,0.257637,0.093278,0,2,1,0.0,0.0,0.0,0.0,0.867473,0.823890,0.823890,-0.697811,-0.311761,-0.177802,-0.208248,1.368423,1.121142,0.734510,0.154609,0.756436,0.364036,9.368566,44.252838,21.371540,1926.730835,2336.613525,2403.396729,15412.594727,12097.913086,20351.759766,2.768578,2.328018,2.825927,205,1,1,1783.964111,0.572177
45478,0.001228,1.000000,52.646694,1.009517,0.008819,0.769089,8.150103,8629.933594,0.122356,0.008543,0.038060,0.286731,0.024846,0,2,0,0.0,0.0,0.0,0.0,0.934495,0.934495,0.719623,-1.020152,-0.269676,-0.300122,-0.450355,0.691030,1.401399,0.964500,0.575982,0.359241,0.336460,8.886332,30.627249,21.443039,402.388214,4380.829102,3906.171631,10589.078125,108756.500000,133513.781250,3.962947,3.904615,4.224580,300,1,1,1769.629395,0.329092
2583,0.000641,0.999931,5.975876,0.298114,0.069880,3.800114,7.624316,3507.376465,0.158135,0.006316,0.158239,0.096123,0.072929,1,2,1,0.0,1.0,0.0,1.0,0.905173,0.562683,0.465120,-0.250856,-0.186523,-0.036454,-0.027879,1.146612,2.169724,1.156981,0.220572,0.475800,0.106540,3.659899,14.310280,4.953084,386.018738,945.195740,2178.675293,4852.797363,6461.723633,40627.386719,3.222986,2.610014,3.618153,186,-99,0,1669.694946,0.206140
29111,0.000361,0.999051,2.052216,0.136910,0.093032,6.564189,1.652357,6301.269531,0.015981,0.014536,0.004173,0.157119,0.071218,1,2,0,1.0,2.0,0.0,3.0,1.000000,0.617369,0.617369,-0.389445,-0.019306,-0.089797,-0.280342,1.098584,0.679145,1.202928,0.164807,0.121844,0.143390,9.387158,7.297283,8.222139,2004.452393,2760.527100,1727.466309,11180.704102,12460.376953,10097.848633,2.403832,2.187777,2.451416,37,4,1,1773.920410,0.760365


In [8]:
domain_adaptation = pd.read_csv('../datasets/domain_adaptation_random.csv').sample(frac=1., random_state=SEED)
domain_adaptation['mass'] = label_prediction['mass'].mean()
domain_adaptation['min_ANNmuon'] = label_prediction['min_ANNmuon'].mean()
domain_adaptation['production'] = label_prediction['production'].mean()
print domain_adaptation.shape
domain_adaptation.head()

(135106, 50)


,LifeTime,dira,FlightDistance,FlightDistanceError,IP,IPSig,VertexChi2,pt,DOCAone,DOCAtwo,DOCAthree,IP_p0p2,IP_p1p2,isolationa,isolationb,isolationc,isolationd,isolatione,isolationf,iso,CDF1,CDF2,CDF3,ISO_SumBDT,p0_IsoBDT,p1_IsoBDT,p2_IsoBDT,p0_track_Chi2Dof,p1_track_Chi2Dof,p2_track_Chi2Dof,p0_IP,p1_IP,p2_IP,p0_IPSig,p1_IPSig,p2_IPSig,p0_pt,p1_pt,p2_pt,p0_p,p1_p,p2_p,p0_eta,p1_eta,p2_eta,SPDhits,domain,mass,min_ANNmuon,production
21840,0.001173,0.999793,7.309732,0.225475,0.144760,8.930744,4.965856,4945.793457,0.019389,0.015251,0.037411,0.204333,0.249398,1,5,3,1.0,2.0,2.0,5.0,1.000000,0.643805,0.270755,-0.423511,-0.131208,-0.115879,-0.176424,1.225634,1.368773,1.647525,0.339441,0.228714,0.454121,10.899674,11.228765,24.274082,885.767273,1630.131714,2696.752686,8884.096680,12979.206055,17545.503906,2.996217,2.763868,2.559938,572,0,1779.389403,0.488508,-36.657765
123606,0.002917,0.999991,13.179792,0.124931,0.056593,4.327537,7.195140,4935.144531,0.039205,0.037555,0.006955,0.484230,0.368756,0,0,3,0.0,0.0,0.0,0.0,0.816136,0.465712,0.422229,-1.106902,-0.238089,-0.367395,-0.501418,1.560585,1.122543,1.224513,1.538280,1.037786,0.150551,78.596626,32.999645,10.717737,1042.821045,1768.838745,2451.449219,4810.159668,10328.353516,13843.159180,2.209985,2.450302,2.416325,224,1,1779.389403,0.488508,-36.657765
45478,0.001228,1.000000,52.646694,1.009517,0.008819,0.769089,8.150103,8629.933594,0.122356,0.008543,0.038060,0.286731,0.024846,0,2,0,0.0,0.0,0.0,0.0,0.934495,0.934495,0.719623,-1.020152,-0.269676,-0.300122,-0.450355,0.691030,1.401399,0.964500,0.575982,0.359241,0.336460,8.886332,30.627249,21.443039,402.388214,4380.829102,3906.171631,10589.078125,108756.500000,133513.781250,3.962947,3.904615,4.224580,300,0,1779.389403,0.488508,-36.657765
70136,0.000888,0.999911,3.751880,0.205613,0.049413,3.308183,1.282461,3560.137695,0.024591,0.024395,0.014363,0.055511,0.039771,5,6,11,1.0,1.0,0.0,2.0,0.418602,0.334923,0.327201,-0.453168,-0.152291,-0.124454,-0.176424,0.773729,1.650806,1.349887,0.622369,0.198982,0.136246,13.011620,10.056454,6.481815,387.275085,1425.018677,2228.083008,3266.975586,9333.081055,15420.416016,2.822100,2.566648,2.622437,287,1,1779.389403,0.488508,-36.657765
96664,0.000629,0.999974,17.316277,1.145542,0.151398,9.436165,3.849268,5634.192871,0.092609,0.020451,0.080179,0.139207,0.153073,5,9,8,3.0,2.0,0.0,5.0,0.570900,0.390120,0.358518,-0.184568,-0.050768,-0.066900,-0.066900,1.153638,0.860187,1.109907,0.262164,0.324961,0.237144,5.828904,3.597851,15.289275,2203.238281,490.422485,3105.210693,85277.132812,20857.156250,76515.773438,4.348958,4.443194,3.897151,402,1,1779.389403,0.488508,-36.657765


In [9]:
check_agreement = pd.read_csv('../datasets/check_agreement.csv.zip')
check_agreement['mass'] = label_prediction['mass'].mean()
check_agreement['min_ANNmuon'] = label_prediction['min_ANNmuon'].mean()
check_agreement['production'] = label_prediction['production'].mean()

print check_agreement.shape
check_agreement.head()

(331147, 52)


,id,LifeTime,dira,FlightDistance,FlightDistanceError,IP,IPSig,VertexChi2,pt,DOCAone,DOCAtwo,DOCAthree,IP_p0p2,IP_p1p2,isolationa,isolationb,isolationc,isolationd,isolatione,isolationf,iso,CDF1,CDF2,CDF3,ISO_SumBDT,p0_IsoBDT,p1_IsoBDT,p2_IsoBDT,p0_track_Chi2Dof,p1_track_Chi2Dof,p2_track_Chi2Dof,p0_IP,p1_IP,p2_IP,p0_IPSig,p1_IPSig,p2_IPSig,p0_pt,p1_pt,p2_pt,p0_p,p1_p,p2_p,p0_eta,p1_eta,p2_eta,SPDhits,signal,weight,mass,min_ANNmuon,production
0,15347063,0.001451,0.999964,6.945030,0.229196,0.058117,2.961298,7.953543,2251.611816,0.082219,0.084005,0.066887,0.185107,0.214719,8,6,1,2.0,1.0,1.0,4.0,0.732076,0.492269,0.179091,-0.207475,-0.019306,-0.089797,-0.098372,0.606178,0.862549,1.487057,0.483199,0.474925,0.426797,24.701061,10.732132,8.853514,1438.064697,468.645721,834.562378,10392.814453,6380.673828,15195.594727,2.666142,3.302978,3.594246,512,0,-0.307813,1779.389403,0.488508,-36.657765
1,14383299,0.000679,0.999818,9.468235,0.517488,0.189683,14.413060,7.141451,10594.470703,0.007983,0.044154,0.001321,0.039357,0.217507,5,6,17,1.0,1.0,1.0,3.0,0.802508,0.605835,0.584701,-0.659644,-0.278330,-0.186370,-0.194944,1.900118,1.073474,1.336784,0.712242,0.260311,0.123877,11.312134,16.435398,7.737038,316.791351,7547.703613,2861.309814,3174.356934,64480.023438,23134.953125,2.995265,2.834816,2.779366,552,0,-0.331421,1779.389403,0.488508,-36.657765
2,7382797,0.003027,0.999847,13.280714,0.219291,0.231709,11.973175,4.778880,2502.196289,0.045085,0.106614,0.005850,0.335788,0.885080,2,2,1,0.0,0.0,1.0,1.0,0.682607,0.682607,0.295038,-0.399239,-0.115879,-0.131069,-0.152291,0.660675,1.683084,0.798658,0.381544,1.163556,1.290409,16.435801,20.686119,44.521961,1887.477905,317.579529,932.128235,15219.761719,3921.181641,10180.791016,2.776633,3.204923,3.081832,318,0,-0.382215,1779.389403,0.488508,-36.657765
3,6751065,0.000810,0.999998,5.166821,0.167886,0.011298,0.891142,5.528002,5097.813965,0.055115,0.038642,0.003864,0.076522,0.068347,4,4,3,0.0,0.0,0.0,0.0,0.533615,0.533615,0.533615,-0.821041,-0.208248,-0.177802,-0.434991,0.770563,1.093031,0.938619,0.564650,0.164411,0.166646,24.878387,7.873435,9.630725,975.041687,1650.837524,2617.248291,4365.082520,13221.149414,24291.875000,2.179345,2.769762,2.918251,290,0,1.465194,1779.389403,0.488508,-36.657765
4,9439580,0.000706,0.999896,10.897236,0.284975,0.160511,16.367550,8.670339,20388.097656,0.015587,0.020872,0.014612,0.249906,0.139937,0,1,0,0.0,0.0,0.0,0.0,0.926410,0.926410,0.926410,-1.116815,-0.328938,-0.443564,-0.344313,1.080559,1.471946,1.123868,0.373736,0.230584,0.112430,28.557213,18.738485,7.389726,6035.000977,9657.492188,4763.682617,27463.011719,46903.394531,24241.628906,2.196114,2.262732,2.310401,45,0,-0.477084,1779.389403,0.488508,-36.657765


In [10]:
Xd, yd = domain_adaptation[features].values, domain_adaptation.domain.values
Xt, yt = label_prediction[features].values, label_prediction.signal.values

Xd.shape, yd.shape, Xt.shape, yt.shape

((135106, 49), (135106,), (135106, 49), (135106,))

In [12]:
def batch_gen(batches, id_array, data, labels):
    for batch_index, (batch_start, batch_end) in enumerate(batches):
        batch_ids = id_array[batch_start:batch_end]
        if labels is not None:
            yield data[batch_ids], labels[batch_ids]
        else:
            yield data[batch_ids]
        np.random.shuffle(id_array)


def evaluate_dann(X_test, batch_size):
    """Predict batch by batch."""
    size = batch_size / 2
    num_batches = X_test.shape[0] / size
    acc = 0
    for i in range(0, num_batches):
        _, prob = dann_model.predict_on_batch(X_test[i * size:i * size + size])
        predictions = np.argmax(prob, axis=1)
        actual = np.argmax(y_test[i * size:i * size + size], axis=1)
        acc += float(np.sum((predictions == actual))) / size
    return acc / num_batches


# Model parameters

batch_size = 128
nb_epoch = 15
nb_classes = 10
img_rows, img_cols = 28, 28
nb_filters = 32
nb_pool = 2
nb_conv = 5

_TRAIN = K.variable(1, dtype='uint8')

# Prep source data
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

# Prep target data
mnistm = mnist_m.load_data()
XT_test = np.swapaxes(np.swapaxes(mnistm['test'], 1, 3), 2, 3)
XT_train = np.swapaxes(np.swapaxes(mnistm['train'], 1, 3), 2, 3)

X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = np.concatenate([X_train, X_train, X_train], axis=1)
X_test = np.concatenate([X_test, X_test, X_test], axis=1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

XT_train = XT_train.astype('float32')
XT_test = XT_test.astype('float32')
XT_train /= 255
XT_test /= 255

domain_labels = np.vstack([np.tile([0, 1], [batch_size / 2, 1]),
                           np.tile([1., 0.], [batch_size / 2, 1])])

# Created mixed dataset for TSNE visualization
num_test = 500
combined_test_imgs = np.vstack([X_test[:num_test], XT_test[:num_test]])
combined_test_labels = np.vstack([y_test[:num_test], y_test[:num_test]])
combined_test_domain = np.vstack([np.tile([1., 0.], [num_test, 1]),
                                 np.tile([0., 1.], [num_test, 1])])


class DANNBuilder(object):
    def __init__(self):
        self.model = None
        self.net = None
        self.domain_invariant_features = None
        self.grl = None
        self.opt = SGD()

    def _build_feature_extractor(self, model_input):
        '''Build segment of net for feature extraction.'''
        net = Convolution2D(nb_filters, nb_conv, nb_conv,
                            border_mode='valid',
                            activation='relu')(model_input)
        net = Convolution2D(nb_filters, nb_conv, nb_conv,
                            activation='relu')(net)
        net = MaxPooling2D(pool_size=(nb_pool, nb_pool))(net)
        net = Dropout(0.5)(net)
        net = Flatten()(net)
        self.domain_invariant_features = net
        return net

    def _build_classifier(self, model_input):
        net = Dense(128, activation='relu')(model_input)
        net = Dropout(0.5)(net)
        net = Dense(nb_classes, activation='softmax',
                    name='classifier_output')(net)
        return net

    def build_source_model(self, main_input, plot_model=False):
        net = self._build_feature_extractor(main_input)
        net = self._build_classifier(net)
        model = Model(input=main_input, output=net)
        if plot_model:
            plot(model, show_shapes=True)
        model.compile(loss={'classifier_output': 'categorical_crossentropy'},
                      optimizer=self.opt, metrics=['accuracy'])
        return model

    def build_dann_model(self, main_input, hp_lambda, plot_model=False):
        net = self._build_feature_extractor(main_input)
        self.grl = GradientReversalLayer
        branch = self.grl(net, hp_lambda)
        branch = Dense(128, activation='relu')(branch)
        branch = Dropout(0.1)(branch)
        branch = Dense(2, activation='softmax', name='domain_output')(branch)

        # When building DANN model, route first half of batch (source examples)
        # to domain classifier, and route full batch (half source, half target)
        # to the domain classifier.
        net = Lambda(lambda x: K.switch(K.learning_phase(), x[:int(batch_size / 2), :], x, lazy=True),
                     output_shape=lambda x: ((batch_size / 2,) + x[1:]))(net)

        net = self._build_classifier(net)
        model = Model(input=main_input, output=[branch, net])
        if plot_model:
            plot(model, show_shapes=True)
        model.compile(loss={'classifier_output': 'categorical_crossentropy',
                      'domain_output': 'categorical_crossentropy'},
                      optimizer=self.opt, metrics=['accuracy'])
        return model
    
main_input = Input(shape=(3, img_rows, img_cols), name='main_input')

builder = DANNBuilder()
src_model = builder.build_source_model(main_input)
src_vis = builder.build_tsne_model(main_input)

hp_lambda = K.variable(1.0)
dann_model = builder.build_dann_model(main_input, hp_lambda)
print('Training source only model')
src_model.fit(X_train, y_train, batch_size=64, nb_epoch=10, verbose=1,
              validation_data=(X_test, y_test))
print('Evaluating target samples on source-only model')
print('Accuracy: ', src_model.evaluate(XT_test, y_test)[1])

# Broken out training loop for a DANN model.
src_index_arr = np.arange(X_train.shape[0])
target_index_arr = np.arange(XT_train.shape[0])

batches_per_epoch = len(X_train) / batch_size
num_steps = nb_epoch * batches_per_epoch
j = 0

print('Training DANN model')

for i in range(nb_epoch):

    batches = make_batches(X_train.shape[0], batch_size / 2)
    target_batches = make_batches(XT_train.shape[0], batch_size / 2)

    src_gen = batch_gen(batches, src_index_arr, X_train, y_train)
    target_gen = batch_gen(target_batches, target_index_arr, XT_train, None)

    losses = list()
    acc = list()

    print('Epoch ', i)

    for (xb, yb) in src_gen:

        # Update learning rate and gradient multiplier as described in
        # the paper.
        p = float(j) / num_steps
        l = 2. / (1. + np.exp(-10. * p)) - 1
        lr = 0.01 / (1. + 10 * p)**0.75
        hp_lambda = l
        builder.opt.lr = lr

        if xb.shape[0] != batch_size / 2:
            continue

        try:
            xt = target_gen.next()
        except:
            # Regeneration
            target_gen = target_gen(target_batches, target_index_arr, XT_train,
                                    None)

        # Concatenate source and target batch
        xb = np.vstack([xb, xt])

        metrics = dann_model.train_on_batch({'main_input': xb},
                                            {'classifier_output': yb,
                                            'domain_output': domain_labels},
                                            check_batch_dim=False)
        j += 1

print('Evaluating target samples on DANN model')
acc = evaluate_dann(XT_test, batch_size)
print('Accuracy:', acc)
print('Visualizing output of domain invariant features')

NameError: name 'y_train' is not defined